## Projet de MODS206 - Nowcasting and Dynamic Factor Models

### BASTIDE Simon, RISTORCELLI Ulysse, VUJASINOVIC Thomas

Données de fréquentations des gares SNCF:

In [1]:
#imports
import pandas as pd

In [4]:
df=pd.read_csv('databases/famille_SUIVI-DEMANDE-TOURISTIQUE/caractéristiques.csv', sep=';')
df

,idBank,Dernière mise à jour,Périodicité,Série arrêtée,Types d'indicateurs pour le tourisme,Indicateur,Mois de retour du voyage,Nature,Mode de transport,Correction,Destination du voyage,Type d’hébergement,Unité,Puissance,Zone géographique,Motif du voyage
0,010758192,20/05/2022 03:55,Annuelle,oui,NaN,Taux de départ en voyages,Sans objet,Taux,Tous modes de transport confondus,Non corrigé,Toutes destinations confondues,Tous types d’hébergement confondus,%,0.0,France métropolitaine,Voyage personnel
1,010758193,06/06/2023 04:47,Annuelle,non,NaN,Voyages,Sans objet,Valeur absolue,Tous modes de transport confondus,Non corrigé,Toutes destinations confondues,Tous types d’hébergement confondus,Voyages,6.0,France métropolitaine,Tous motifs confondus
2,010758194,06/06/2023 04:47,Annuelle,non,NaN,Voyages,Sans objet,Valeur absolue,Tous modes de transport confondus,Non corrigé,Voyages à destination de la France métropolita...,Tous types d’hébergement confondus,Voyages,6.0,France métropolitaine,Tous motifs confondus
3,010758195,06/06/2023 04:47,Annuelle,non,NaN,Voyages,Sans objet,Valeur absolue,Tous modes de transport confondus,Non corrigé,Voyages à destination de l’Europe (Union Europ...,Tous types d’hébergement confondus,Voyages,6.0,France métropolitaine,Tous motifs confondus
4,010758196,06/06/2023 04:47,Annuelle,non,NaN,Voyages,Sans objet,Valeur absolue,Tous modes de transport confondus,Non corrigé,Voyages à destination de l’étranger hors Europe,Tous types d’hébergement confondus,Voyages,6.0,France métropolitaine,Tous motifs confondus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,R,Révision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,A,Valeur normale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,N,Valeur non significative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,O,Valeur manquante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
